In [ ]:
Predictive question: 
Using CPI, Current account, and unemployment rate to predict Real GNI of 2022 
using regression analysis 2013-2021

Model: regression

Variables:

Real GNI, annually, Canada, 2013-2022
GDP:GNI.csv

CPI, unadjusted, 2013-2022: CPI.csv

Current account, annual, Canada, 2013-2022: Current account.csv

Unemployment rate, age 15 or above, Canada, both sexes, 2013-2022: unemployment.csv